In [1]:
import pickle
import numpy as np
from re import split
import math
from textblob import TextBlob as tb
import nltk
# nltk.download('punkt')

dict_keys(['label', 'outputs', 'text', 'encoding', 'prediction'])
64301
128
64301


In [59]:
data = pickle.load( open( "emotype_v3.p", "rb" ) ) 
print (data[0].keys())
print (len(data))

_,m = data[0]['encoding'].shape
n = len(data)
print (m)
print (n)

dict_keys(['text', 'encoding', 'label', 'outputs', 'prediction'])
80
64
80


In [60]:
label_cluster_dic = {'addiction':0, 'anxiety':1, 'autism':2, 'bipolar':3, 'conversation':4, 'depression':5,
 'happy':6, 'schizophrenia':7}
for i in range(len(data)):
    data[i]['cluster'] = label_cluster_dic[data[i]['label']]

In [7]:
all_text  = []
for i in range(len(data)):
    tmp = data[i]['text'].strip().lower()
    all_text.append(split("[^a-zA-Z']+", tmp))
    
print (len(all_text))

In [12]:
dic = {}
for i in range(len(all_text)):
    for j in all_text[i]:
        if j and dic.get(j,-1)!= 0:
            dic[j] = 0
            
print(len(dic.keys()))
print(dic['you'])

64301


In [28]:
pickle.dump( all_text, open( "all_text.p", "wb" ) )
pickle.dump( dic, open( "dic.p", "wb" ) )

In [29]:
word_list = list(dic.keys())
word_list[1]

'do'

'do'

In [46]:
def find_text_with_cluster(cluster):
    out = []
    for i in range(len(data)):
        if data[i]['cluster'] == cluster:
            out.append(split("[^a-zA-Z']+", data[i]['text'].strip().lower() ))
    return out
def count_word_of_cluster(texts):
    out_dic = dic.fromkeys(dic,0)
    for text in texts:
        for word in text:
            if word: out_dic[word] += 1
    return out_dic
def get_cluster_vectors(word_list,dic_cluster):
    out = []
    for word in word_list:  
        out.append(dic_cluster[word])
    return out
def extract_vectors(cluster):
    text_cluster = find_text_with_cluster(cluster)
    dic_cluster = count_word_of_cluster(text_cluster)
    vector_cluster = get_cluster_vectors(word_list,dic_cluster)
    return vector_cluster

In [24]:
def tf(word, blob):
    '''computes "term frequency" which is the number of times a word appears in a document blob, 
       normalized by dividing by the total number of words in blob. 
       We use TextBlob for breaking up the text into words and getting the word counts.'''
    return blob.words.count(word) / len(blob.words)

def n_containing(word, bloblist):
    '''returns the number of documents containing word. 
    A generator expression is passed to the sum() function.'''
    return sum(1 for blob in bloblist if word in blob.words)

def idf(word, bloblist):
    '''computes "inverse document frequency" or how common a word is among all documents in bloblist. 
       The more common a word is, the lower its idf. 
       We take the ratio of the total number of documents to the number of documents containing word, 
       then take the log of that. Add 1 to the divisor to prevent division by zero.'''
    return math.log(len(bloblist) / (1 + n_containing(word, bloblist)))

def tfidf(word, blob, bloblist):
    ''' computes the TF-IDF score. It's the product of tf and idf.'''
    return tf(word, blob) * idf(word, bloblist)

In [55]:
document1 = tb("one two three")

document2 = tb("two three four")

document3 = tb("three four five")

bloblist = [document1, document2, document3]
for i, blob in enumerate(bloblist):
    print("Top words in document {}".format(i + 1))
    scores = {word: tfidf(word, blob, bloblist) for word in blob.words}
    sorted_words = sorted(scores.items(), key=lambda x: x[1], reverse=True)
    for word, score in sorted_words[:2]:
        print("\tWord: {}, TF-IDF: {}".format(word, round(score, 5)))

Top words in document 1
	Word: one, TF-IDF: 0.13516
	Word: two, TF-IDF: 0.0
Top words in document 2
	Word: two, TF-IDF: 0.0
	Word: four, TF-IDF: 0.0
Top words in document 3
	Word: five, TF-IDF: 0.13516
	Word: four, TF-IDF: 0.0


In [31]:
# sanity check
blob =  tb("one two three")
tf('one', blob)

0.3333333333333333

In [39]:
n_containing('two', bloblist)

2

In [41]:
idf('three', bloblist) # log (3/(1+2))

-0.2876820724517809

In [42]:
math.log(3/4)

-0.2876820724517809

In [65]:
bloblist = []
diagnosis = []
for cluster in list(label_cluster_dic.keys()):
    print(cluster)
    diagnosis.append(cluster)
    cluster_text = find_text_with_cluster(label_cluster_dic[cluster])
    list_of_posts = []
    for post in cluster_text:
        list_of_posts.append(' '.join(post))
    big_string = tb(' '.join(list_of_posts))
    bloblist.append(big_string)

addiction
anxiety
autism
bipolar
conversation
depression
happy
schizophrenia


In [68]:
for i, blob in enumerate(bloblist):
    print("Top words in document {},{}".format(i + 1, diagnosis[i]))
    scores = {word: tfidf(word, blob, bloblist) for word in blob.words}
    sorted_words = sorted(scores.items(), key=lambda x: x[1], reverse=True)
    for word, score in sorted_words[:5]:
        print("\tWord: {}, TF-IDF: {}".format(word, round(score, 5)))

Top words in document 1,addiction
	Word: kick, TF-IDF: 0.00865
	Word: fent, TF-IDF: 0.00576
	Word: puff, TF-IDF: 0.00576
	Word: pot, TF-IDF: 0.00576
	Word: caused, TF-IDF: 0.00576
Top words in document 2,anxiety
	Word: shit, TF-IDF: 0.00825
	Word: public, TF-IDF: 0.00825
	Word: ven, TF-IDF: 0.00825
	Word: anxiety, TF-IDF: 0.00778
	Word: nauseous, TF-IDF: 0.0055
Top words in document 3,autism
	Word: radiohead, TF-IDF: 0.00872
	Word: asperger, TF-IDF: 0.00872
	Word: awkward, TF-IDF: 0.00872
	Word: wants, TF-IDF: 0.00872
	Word: he, TF-IDF: 0.00591
Top words in document 4,bipolar
	Word: bpd, TF-IDF: 0.00947
	Word: bipolar, TF-IDF: 0.00947
	Word: brain, TF-IDF: 0.00632
	Word: fog, TF-IDF: 0.00632
	Word: openly, TF-IDF: 0.00632
Top words in document 5,conversation
	Word: agent, TF-IDF: 0.01132
	Word: mozart, TF-IDF: 0.00849
	Word: asks, TF-IDF: 0.00849
	Word: brit, TF-IDF: 0.00849
	Word: he, TF-IDF: 0.00576
Top words in document 6,depression
	Word: one, TF-IDF: 0.0069
	Word: leave, TF-IDF: 0